---
### Simplified System of linear equations solving for image $X$

Key terms:

$X$ = the ideal image to solve for, assumed to be the "perfect" image we desire

$Y_i$ = a decimated, downsampled, blurred, noisy image based on $X$, assumed to be one of many, where $1 \leq i \leq N$

$L$ = one-dimensional size for image $X$

$M$ = one-dimensional size for each image in $Y$

$N$ = the amount of $Y$ images in use

$D$ = the decimation operator of size $M^2 \times L^2$, assumed identical for all $Y_i$

$U$ = the "undecimation" operator, taken as transpose of $D$

$H$ = the blur operator of size $L^2 \times L^2$, assumed identical for all $Y_i$

$E_i$ = the additive noise matrix of size $M^2 \times M^2$ for matrix $Y_i$

$S$ = the sharpening operator of size $L^2 \times L^2$ (?), i.e., the laplacian operator

$w_i$ = a weighting scalar, related to autocorrelation of image noise per image $Y_i$ (I think?...)

$\beta$ = smoothing factor for solving, relative to sum of $Y_i$ image weights $\sum\limits_{i=1}^{N}w_i$

---

##### Standard library imports

In [1]:
%matplotlib inline

import numpy as np
from numpy.random import default_rng

import matplotlib.pyplot as plt

##### Helper functions and filters

In [2]:
def show_image(image):
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.colorbar()
    plt.show()
    
box_blur = np.full((3, 3), 1/9) # 3x3 box blur
laplacian = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]]) # 3x3 laplacian sharpen

##### Attempting to make sense of the formulation...

In [101]:
L = 4 # X = 4x4 pixels
M = 2 # Y_i = 2x2 pixels
N = 1 # only 1 Y image
β = 0 # no smoothing influence
W = np.ones(N) # no scaled weighting

X = np.arange(1, L**2+1)
Y = np.zeros(M**2)

D = np.zeros((M**2, L**2))
D[:, 0:M] = 1
D[:, M+M:M**2+M] = 1
dist = M**2+M
for r in np.arange(M**2):
    if r & 0x1: # is odd
        roll_dist = ((r-1) * dist) + M
    else: # is even
        roll_dist = ((r-1) * dist) + M 
        
    print(roll_dist)
    D[r] = np.roll(D[r], roll_dist)
# D[0][0:6] = [1, 1, 0, 0, 1, 1]
# D[1][2:8] = [1, 1, 0, 0, 1, 1]
# D[2][8:14] = [1, 1, 0, 0, 1, 1]
# D[3][10:16] = [1, 1, 0, 0, 1, 1]
# D = D * 1/4 # average of 4 neighbours
print(D)

-4
2
8
14
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [100]:
r = 0
print(r & 0x1)

0
